In [1]:
import os

RS_HOST = os.environ["RS_HOST"] if "RS_HOST" in os.environ else "redshift-production.weworkers.io"
RS_PORT = int(os.environ["RS_PORT"]) if "RS_PORT" in os.environ else 5439
RS_DATABASE = os.environ["RS_DATABASE"] if "RS_DATABASE" in os.environ else "analyticdb"

RS_USER = os.environ["RS_USERNAME"] if "RS_USERNAME" in os.environ else "qi.chen2@wework.com"
RS_PASS = os.environ["RS_PASSWORD"] if "RS_PASSWORD" in os.environ else "513846129Cq123!"

RS_URL = os.environ["RS_URL"] if "RS_URL" in os.environ else "redshift://redshift-production.weworkers.io:5439/analyticdb"


In [2]:
class FloorMedia:

    def __init__(self, uuid, floor_id, url, extension, plan_type):
        self.uuid = uuid
        self.floor_id = floor_id
        self.url = url
        self.extension = extension
        self.type = plan_type


In [ ]:
import psycopg2
import requests
import numpy as np

# from credentials import *

connect = psycopg2.connect(dbname=RS_DATABASE, user=RS_USER, password=RS_PASS, host=RS_HOST, port=RS_PORT)
cursor = connect.cursor()

url = "https://sg-bs.wework.com/"

cursor.execute('''SELECT * FROM stargate_modules_pg_public.bs_harvest_floor_media;''')

raw_data = cursor.fetchall()

data = []

# number of floor plans you want to sample
n_image = len(raw_data)
for i in range(n_image):
    row = raw_data[i]
    floor = FloorMedia(row[0], row[1], row[3], row[5], row[8])
    data.append(floor)

count = 0
for floor in data:
    r = requests.get(url + floor.url, stream=True)
    if r.status_code == 200:
        if floor.extension == "png":
            with open("floor_plan_image/"+str(floor.floor_id)+".png", "wb") as f:
#                 print("Writing image for floor " + floor.floor_id)
                for chunk in r:
                    f.write(chunk)
    count += 1
    if count / 100 == 0:
        print(count)


cursor.close()
connect.close()



In [ ]:
5:31  ---> 0
5:41  ---> 265